# Importando as bibliotecas necessárias

In [40]:

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import klib
import seaborn as sns
from scipy.stats import linregress
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from xgboost import plot_importance
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, make_scorer
import shap
import xgboost as xgb
from sklearn.preprocessing import RobustScaler

# Carregando os arquivos em CSV

In [2]:
c = pd.read_csv('base_dados/base_clientes.csv')
hc = pd.read_csv('base_dados/base_hist_churn.csv')
p = pd.read_csv('base_dados/base_precos.csv')
pd.set_option('display.max_columns', None)

# Exploração de dados

## Puxando as informações da tabela de clientes

In [ ]:
c.info()

## Substituindo os tipos da tabela de clientes "objects" por "categóricos" e "float64", "int64" por "numéricos"

|    | Column                  | Non-Null Count | Dtype       |
|---:|:------------------------|:--------------:|:------------|
|  0 | id                      | 20120          | categórico  |
|  1 | activity_new            | 8221           | categórico  |
|  2 | campaign_disc_ele       | 0              | numérico    |
|  3 | channel_sales           | 14846          | categórico  |
|  4 | cons_12m                | 20120          | numérico    |
|  5 | cons_gas_12m            | 20120          | numérico    |
|  6 | cons_last_month         | 20120          | numérico    |
|  7 | date_activ              | 20120          | categórico  |
|  8 | date_end                | 20118          | categórico  |
|  9 | date_first_activ        | 4385           | categórico  |
| 10 | date_modif_prod         | 19918          | categórico  |
| 11 | date_renewal            | 20076          | categórico  |
| 12 | forecast_base_bill_ele  | 4385           | numérico    |
| 13 | forecast_base_bill_year | 4385           | numérico    |
| 14 | forecast_bill_12m       | 4385           | numérico    |
| 15 | forecast_cons           | 4385           | numérico    |
| 16 | forecast_cons_12m       | 20120          | numérico    |
| 17 | forecast_cons_year      | 20120          | numérico    |
| 18 | forecast_discount_energy| 19970          | numérico    |
| 19 | forecast_meter_rent_12m | 20120          | numérico    |
| 20 | forecast_price_energy_p1| 19970          | numérico    |
| 21 | forecast_price_energy_p2| 19970          | numérico    |
| 22 | forecast_price_pow_p1   | 19970          | numérico    |
| 23 | has_gas                 | 20120          | categórico  |
| 24 | imp_cons                | 20120          | numérico    |
| 25 | margin_gross_pow_ele    | 20105          | numérico    |
| 26 | margin_net_pow_ele      | 20105          | numérico    |
| 27 | nb_prod_act             | 20120          | numérico    |
| 28 | net_margin              | 20102          | numérico    |
| 29 | num_years_antig         | 20120          | numérico    |
| 30 | origin_up               | 20006          | categórico  |
| 31 | pow_max                 | 20116          | numérico    |

## Puxando as informações da tabela de churn

In [ ]:
hc.info()

## Substituindo os tipos da tabela de churn objects por "categóricos" e "float64", "int64" por "numéricos"

|    | Column | Non-Null Count | Dtype   |
|---:|:-------|:--------------:|:-------:|
|  0 | id     | 16096          | categórico  |
|  1 | churn  | 16096          | numérico   |

## Puxando as informações da tabela de Preços

In [ ]:
p.info()

## Substituindo os tipos da tabela de preços objects por "categóricos" e "float64", "int64" por "numéricos"

|    | Column      | Non-Null Count | Dtype    |
|---:|:------------|:--------------:|:--------:|
|  0 | id          | 241238         | categórico   |
|  1 | price_date  | 241238         | categórico   |
|  2 | price_p1_var| 239577         | numérico  |
|  3 | price_p2_var| 239577         | numérico  |
|  4 | price_p3_var| 239577         | numérico  |
|  5 | price_p1_fix| 239577         | numérico  |
|  6 | price_p2_fix| 239577         | numérico  |
|  7 | price_p3_fix| 239577         | numérico  |

## Estatística descritiva das colunas da tabela de clientes

In [ ]:
c.describe()

As estatísticas da tabela de clientes forneceram informações sobre a distribuição dos dados e auxiliam a entender a tendência central, a dispersão e os valores extremos em cada variável. Com base nessas estatísticas, retiram-se várias conclusões sobre seus dados, como a presença de outliers, a faixa de valores típicos, relação entre diferentes variáveis, também serve de orientação para definir ações estratégicas e relevar insights importantes. Ademais foi observado que os valores negativados (na linha 'min') são erros de input que serão analisados e tratados futuramente pela equipe.

## Estatística descritiva das colunas da tabela de churn

In [ ]:
hc.describe()

Essa análise sugere que a maioria das amostras tem um valor de churn igual a 0 (sem cancelamento), já que tanto o primeiro quanto o terceiro quartil (conceito estatístico utilizado para dividir um conjunto de dados ordenados em quatro partes iguais) são 0. No entanto, a média de 0.099 sugere que cerca de 9.9% das amostras têm um valor de churn igual a 1 (houve cancelamento).

## Estatística descritiva das colunas da tabela de Preços

In [ ]:
p.describe()

Essas estatísticas fornecem uma visão geral sobre a distribuição e os intervalos de valores para cada variável. Esses valores são úteis para entender a variação e a tendência central dos preços associados a diferentes componentes, também, para identificar possíveis valores extremos ou anomalias nos dados.

## Tabela de clientes com valores faltando

In [ ]:
klib.missingval_plot(c)

## Tabela de churn com valores faltando

In [ ]:
klib.missingval_plot(hc)

## Tabela de preços com valores faltando

In [ ]:
klib.missingval_plot(p)

## Matriz de relação entre as colunas

Entender a relação entre as colunas é fundamental para decifrar como as variáveis interagem. Essa compreensão desempenha um papel vital na identificação das colunas mais pertinentes à previsão. Isso também facilita a distinção de colunas que não são relevantes. Embora nem todas sejam essenciais, algumas podem ser dispensadas quando se trata da previsão do churn. Ao reconhecer as relações, conseguimos rapidamente determinar quais colunas podem ser descartadas. Além disso, ao identificar quais têm uma forte ligação com a variável alvo, ganhamos insights valiosos sobre onde direcionar nossos esforços de otimização.

In [ ]:
#Gera um gráfico da matriz de correlação entre as colunas.
klib.corr_plot(c)

## Gráfico de relação entre forecast_cons e imp_cons

A relação entre o preço atual do consumo de energia (imp_cons) e a previsão de consumo elétrico para o próximo mês (forecast_cons) é forte e relevante porque os valores são muito parecidos. Conclui-se então que ao interpretar essa tabela que uma coluna pode se tornar irrelevante ao entender que os valores são iguais e que há existência de outliers.

In [ ]:
#Relação entre as colunas forecast_cons e imp_cons
plt.scatter(c['forecast_cons'], c['imp_cons'])
# Adicionando título e legendas
plt.title("Relação entre forecast_cons e imp_cons")
plt.xlabel("forecast_cons")
plt.ylabel("imp_cons")

## Gráfico de relação entre forecast_bill_12m e imp_cons

A relação entre o preço atual do consumo de energia (imp_cons) e a previsão da fatura de consumo elétrico para os próximos 12 meses (forecast_bill_12m) é relevante porque os valores se concentram numa parte, podem ser descritos numa regressão linear e com presença de outliers.

In [ ]:
#Relação entre as colunas forecast_bill_12m e imp_cons
plt.scatter(c['forecast_bill_12m'], c['imp_cons'])
# Adicionando título e legendas
plt.title("Relação entre forecast_bill_12m e imp_cons")
plt.xlabel("forecast_bill_12m")
plt.ylabel("imp_cons")

## Gráfico de relação entre forecast_base_bill_year e forecast_base_bill_ele


A relação entre o valor base da fatura de eletricidade prevista para o próximo mês (forecast_base_bill_ele) e o valor base da fatura de eletricidade prevista para o ano (forecast_base_bill_year) é forte e relevante porque os valores são muito parecidos e no final poderemos escolher um deles porque se escolhermos os dois poderemos enviesar o modelo.

In [ ]:
#Relação entre as colunas forecast_price_energy_p1 e forecast_meter_rent_12m
plt.scatter(c['forecast_base_bill_ele'], c['forecast_base_bill_year'])
# Adicionando título e legendas
plt.title("Relação entre forecast_base_bill_ele e forecast_base_bill_year")
plt.xlabel("forecast_base_bill_ele")
plt.ylabel("forecast_base_bill_year")

# 1 Limpeza dos dados e engenharia de features

### 1.1 Engenharia de Features na tabela de preços

<b>Calculando a média das colunas de preço</b>

Foi identificado que seria útil ter a média dos preços variáveis e fixos para cada entrada. Assim, criamos duas novas colunas "total_var_mean" e "total_fix_mean" para armazenar essas médias.

In [3]:
# Calculate the mean for variable prices
p['total_var_mean'] = p[['price_p1_var', 'price_p2_var', 'price_p3_var']].mean(axis=1)

# Calculate the mean for fixed prices
p['total_fix_mean'] = p[['price_p1_fix', 'price_p2_fix', 'price_p3_fix']].mean(axis=1)

<b>Variação percentual das médias de preços</b>

Para entender a evolução das médias dos preços ao longo do tempo para cada ID, calculamos a variação percentual das médias dos preços variáveis e fixos. Criamos as colunas "variation_total_var" e "variation_total_fix" para armazenar essas variações percentuais.

In [4]:
# Calculate the percentage change for the mean of variable prices
p['variation_total_var'] = p.groupby('id')['total_var_mean'].pct_change().fillna(0) * 100

# Calculate the percentage change for the mean of fixed prices
p['variation_total_fix'] = p.groupby('id')['total_fix_mean'].pct_change().fillna(0) * 100

<b>Identificando as maiores variações percentuais por ID</b>

Para cada ID, identificamos as maiores variações percentuais observadas para os preços variáveis e fixos. Isso nos ajuda a entender quais IDs tiveram as mudanças mais significativas em termos de preços.

Primeiro, filtramos as entradas que não possuem uma variação percentual de 0 para a coluna 'variation_total_var'. Em seguida, para cada ID, calculamos a máxima variação percentual tanto para os preços variáveis quanto para os fixos.

In [5]:
highest_variation = p[p['variation_total_var'] != 0].groupby('id')[['variation_total_var', 'variation_total_fix']].max().reset_index()
highest_variation.columns = ['id', 'highest_variation_var', 'highest_variation_fix']

Posteriormente, integramos essa informação ao dataframe original p, para ter um conjunto de dados que contém tanto as informações originais quanto as máximas variações percentuais. Finalmente, substituímos os valores faltantes (NaN) por 0.

In [6]:
data_with_highest_variation = pd.merge(p, highest_variation, on='id', how='left')
data_with_highest_variation.fillna(0, inplace=True)

Isso garante que não haja problemas com valores faltantes no novo dataframe data_with_highest_variation.

<b>Computando o desvio padrão das variações percentuais por ID</b>

Para obter insights adicionais sobre a volatilidade das variações percentuais para cada ID, calculamos o desvio padrão (uma medida de dispersão) das variações tanto para os preços variáveis quanto para os fixos. Este cálculo ajuda a entender quão variadas foram as mudanças de preços para cada ID ao longo do tempo.

Inicialmente, filtramos as entradas que não têm uma variação percentual de 0 para a coluna 'variation_total_var'. Em seguida, para cada ID, calculamos o desvio padrão das variações percentuais.

In [7]:
std_variation = data_with_highest_variation[data_with_highest_variation['variation_total_var'] != 0].groupby('id')[['variation_total_var', 'variation_total_fix']].std().reset_index()
std_variation.columns = ['id', 'std_variation_var', 'std_variation_fix']

Depois disso, integramos essa informação de desvio padrão ao dataframe data_with_highest_variation, gerando um novo dataframe chamado data_with_std_variation. Esta integração garante que as informações originais e os cálculos estatísticos estejam todos no mesmo lugar.

In [8]:
data_with_std_variation = pd.merge(data_with_highest_variation, std_variation, on='id', how='left')
data_with_std_variation.fillna(0, inplace=True)

<b>Computando a média das variações percentuais por ID</b>

Após avaliar o desvio padrão, decidimos também analisar a média das variações percentuais para cada ID. Isso nos fornece uma visão sobre o comportamento médio das variações dos preços ao longo do tempo para cada ID.

Primeiro, filtramos novamente as entradas que não possuem uma variação percentual de 0 para a coluna 'variation_total_var'. Em seguida, para cada ID, calculamos a média das variações percentuais.

In [9]:
mean_variation = data_with_std_variation[data_with_std_variation['variation_total_var'] != 0].groupby('id')[['variation_total_var', 'variation_total_fix']].mean().reset_index()

Depois, integramos essa média das variações percentuais ao dataframe data_with_std_variation para formar um novo dataframe chamado data_with_mean_std_variation. A integração é feita usando o método merge, e para distinguir as colunas originais das colunas com a média, usamos o parâmetro suffixes para adicionar o sufixo '_mean' às colunas com os valores médios.

In [10]:
data_with_mean_std_variation = pd.merge(data_with_std_variation, mean_variation, on='id', how='left', suffixes=('', '_mean'))

<b>Computando o coeficiente de variação das variações percentuais</b>

Para entender a relativa dispersão das variações percentuais, optamos por calcular o coeficiente de variação (CV) para cada ID. O coeficiente de variação é uma medida estatística que expressa a variação dos dados em relação à média, permitindo comparar a dispersão entre diferentes conjuntos de dados.

O coeficiente de variação é calculado dividindo o desvio padrão pela média e, em seguida, multiplicando por 100.

Computamos o CV para as variações percentuais dos preços variáveis e fixos:

In [11]:
data_with_mean_std_variation['cv_variation_var'] = (data_with_mean_std_variation['std_variation_var'] / data_with_mean_std_variation['variation_total_var_mean']) * 100
data_with_mean_std_variation['cv_variation_fix'] = (data_with_mean_std_variation['std_variation_fix'] / data_with_mean_std_variation['variation_total_fix_mean']) * 100

Finalmente, para garantir a integridade do dataframe, substituímos quaisquer valores faltantes (NaN) por 0:

In [12]:
data_with_mean_std_variation.fillna(0, inplace=True)

Deste modo, temos uma visão consolidada da dispersão relativa das variações percentuais para cada ID, permitindo análises mais aprofundadas sobre a estabilidade dos preços ao longo do tempo.

<b>Atribuindo os dados processados ao DataFrame p</b>

Após todos os cálculos e transformações, decidimos atualizar o DataFrame p para que ele contenha as informações consolidadas de data_with_mean_std_variation. Dessa forma, p agora é a versão atualizada do conjunto de dados, incluindo cálculos de desvio padrão, médias e coeficientes de variação.

In [13]:
p = data_with_mean_std_variation

<b>Removendo a coluna 'price_date' e duplicatas</b>

Antes de avançarmos com análises ou outras operações no DataFrame p, decidimos fazer uma limpeza adicional.

In [14]:
p = p.drop('price_date', axis=1).drop_duplicates()

<b>Processamento e Extração de Características</b>

Calculamos a soma dos valores para cada período (var e fix)

In [15]:
# Calculating the sum of values for each period (var and fix)
p['total_var'] = p[['price_p1_var', 'price_p2_var', 'price_p3_var']].sum(axis=1)
p['total_fix'] = p[['price_p1_fix', 'price_p2_fix', 'price_p3_fix']].sum(axis=1)

Criamos indicadores booleanos para o consumo de energia em cada período (var e fix)

In [16]:
# Creating boolean indicators for energy consumption in each period (var and fix)
for period in ['p1', 'p2', 'p3']:
    p[f'{period}_var'] = (p[f'price_{period}_var'] > 0).astype(int)
    p[f'{period}_fix'] = (p[f'price_{period}_fix'] > 0).astype(int)

Agrupamos por 'id' e pegamos a mediana das novas características

In [17]:
# Grouping by 'id' and taking the median of the new features
reshaped_data = p.groupby('id').agg({
    'total_var': 'median',
    'total_fix': 'median',
    'total_var_mean': 'median',
    'total_fix_mean': 'median',
    'highest_variation_var': 'median',
    'highest_variation_fix': 'median',
    'std_variation_var': 'median',
    'std_variation_fix': 'median',
    'cv_variation_var': 'median',
    'cv_variation_fix': 'median',
    'p1_var': 'median',
    'p2_var': 'median',
    'p3_var': 'median',
    'p1_fix': 'median',
    'p2_fix': 'median',
    'p3_fix': 'median'
}).reset_index()

Calculamos o consumo de kwh

In [18]:
# Checking the first few rows of the reshaped data
p = reshaped_data

p['kwh_consumed'] = p['total_fix'] / p['total_var']

Substituímos valores infinitos pelo valor NaN

In [19]:
p['kwh_consumed'].replace([float('inf'), -float('inf')], float('nan'), inplace=True)

In [20]:
p

,id,total_var,total_fix,total_var_mean,total_fix_mean,highest_variation_var,highest_variation_fix,std_variation_var,std_variation_fix,cv_variation_var,cv_variation_fix,p1_var,p2_var,p3_var,p1_fix,p2_fix,p3_fix,kwh_consumed
0,0002203ffbb812588b632b9e628cc38d,0.300305,81.457770,0.100102,27.152590,2.456503,0.401607,4.583068,0.283979,-584.413926,141.421356,1.0,1.0,1.0,1.0,1.0,1.0,271.250129
1,000381698491fec6983d55828fe5ada6,0.259952,104.031888,0.086651,34.677296,-1.733217,0.401603,1.198079,0.283976,-46.430200,141.421356,1.0,1.0,1.0,1.0,1.0,1.0,400.196529
2,0004351ebdd665e6ee664792efc4fd13,0.148047,44.444710,0.049349,14.814903,0.241815,0.401606,2.297003,0.283979,-166.159164,141.421356,1.0,0.0,0.0,1.0,0.0,0.0,300.206758
3,000861c9aa2a0784b8d83f8b5b95d010,0.306110,81.457770,0.102037,27.152590,1.528535,0.401607,3.898444,0.283979,-317.441997,141.421356,1.0,1.0,1.0,1.0,1.0,1.0,266.106204
4,0010bcc39e42b3c2131ed2ce55246e3c,0.201280,45.944710,0.067093,15.314903,34.429721,3.374980,20.162985,1.948546,179.349786,173.205081,1.0,0.0,0.0,1.0,0.0,0.0,228.262669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,ffef185810e44254c3a4c6395e6b4d8a,0.387876,82.625940,0.129292,27.541980,1.792068,0.000000,15.134446,0.816266,-154.280088,-173.205081,1.0,1.0,1.0,1.0,1.0,1.0,213.021533
20116,fffac626da707b1b5ab11e8431a4d0a2,0.145047,44.266930,0.048349,14.755643,0.473806,0.401609,2.455045,0.283980,-194.509338,141.421356,1.0,0.0,0.0,1.0,0.0,0.0,305.190248
20117,fffc0cacd305dd51f316424bbb08d1bd,0.382048,82.957771,0.127349,27.652590,1.817459,0.401606,2.619313,0.231868,-237.108682,173.205081,1.0,1.0,1.0,1.0,1.0,1.0,217.139656
20118,fffe4f5646aa39c7f97f95ae2679ce64,0.294097,81.131934,0.098032,27.043978,-0.311122,0.401613,2.637215,0.283983,-121.200308,141.421356,1.0,1.0,1.0,1.0,1.0,1.0,275.867942


### 1.2 Preenchendo os valores vazios da tabela forecast_base_bill_ele

<b>Imputação com Regressão Linear</b>

Nesta seção, estamos usando a regressão linear para prever e imputar valores faltantes nas colunas forecast_base_bill_ele e forecast_base_bill_year.

Selecionamos as colunas relevantes (forecast_base_bill_ele e imp_cons) e removemos os registros com valores faltantes

In [20]:
regression_data = c[['forecast_base_bill_ele', 'imp_cons']].dropna()

Definimos as variáveis independentes (X) e dependentes (y) para o modelo

In [21]:
X = regression_data[['imp_cons']]
y = regression_data['forecast_base_bill_ele']

Treinamos um modelo de regressão linear com os dados

In [22]:
model = LinearRegression()
model.fit(X, y)

LinearRegression()

Após treinar o modelo, obtemos a inclinação (coeficiente) e o intercepto do modelo

In [23]:
slope = model.coef_[0]
intercept = model.intercept_

Usamos o modelo treinado para prever e imputar os valores faltantes em ambas as colunas

In [24]:
c['forecast_base_bill_ele'].fillna(c['imp_cons'] * slope + intercept, inplace=True)
c['forecast_base_bill_year'].fillna(c['imp_cons'] * slope + intercept, inplace=True)


Verificamos quantos valores faltantes ainda estão presentes nas colunas após a imputação

In [25]:
missing_values_after_imputation = c[['forecast_base_bill_ele', 'forecast_base_bill_year']].isnull().sum()
missing_values_after_imputation

forecast_base_bill_ele     0
forecast_base_bill_year    0
dtype: int64

Esta abordagem nos permite lidar com valores faltantes, usando a relação linear entre as colunas forecast_base_bill_ele e imp_cons para imputar valores faltantes de maneira informada.

### 1.3 Merge na tabela de preços e clientes para limpeza e transformação dos dados

<b>Limpeza de dados</b>

O grupo identificou que as colunas de "campaign_disc_ele" e "date_firts_activ" estão com muitos dados faltantes, portanto estamos deletando essas colunas para que elas não atrapalhem o modelo.

In [26]:
# merge the dataframes
pc = c.merge(p, how='inner', on='id')

# drop the columns campaign_disc_ele and date_first_activ
pc = pc.drop(['campaign_disc_ele','date_first_activ'], axis=1)
pc

,id,activity_new,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_base_bill_ele,forecast_base_bill_year,forecast_bill_12m,forecast_cons,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_p1,forecast_price_energy_p2,forecast_price_pow_p1,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,total_var,total_fix,total_var_mean,total_fix_mean,highest_variation_var,highest_variation_fix,std_variation_var,std_variation_fix,cv_variation_var,cv_variation_fix,p1_var,p2_var,p3_var,p1_fix,p2_fix,p3_fix,kwh_consumed
0,ed18c9981ce183ef347246cdcc55f743,bwpaswkpcilmlklklcapcwwumwaodaoo,foosdfpfkusacimwkcsosbicdxkicaua,42132,0,3215,06/09/2017,06/09/2022,27/08/2021,07/09/2021,451.048858,451.048858,NaN,NaN,4001.08,3215,0.0,130.43,0.114603,0.098261,40.606701,f,296.90,9.63,9.63,1,488.25,4,lxidpiddsbxsbosboudacockeimpuepw,20.000,0.293558,81.457770,0.097853,27.152590,0.596262,0.000000,2.082828,0.000000,-119.379663,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,277.484415
1,9c341e73685fcff6e3527ebd51da3fe5,NaN,foosdfpfkusacimwkcsosbicdxkicaua,3537,0,563,13/12/2016,31/12/2022,13/12/2016,01/01/2022,172.527461,172.527461,NaN,NaN,518.22,563,0.0,23.20,0.142174,0.000000,44.444710,f,80.66,0.12,0.12,1,29.18,5,lxidpiddsbxsbosboudacockeimpuepw,15.000,0.143620,44.444710,0.047873,14.814903,-3.013195,0.000000,0.000000,0.000000,-0.000000,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,309.460451
2,97dd64b33716aadc4e22c4ab85d1c7a4,NaN,foosdfpfkusacimwkcsosbicdxkicaua,6543,0,399,04/09/2016,04/09/2022,27/08/2021,05/09/2021,117.464615,117.464615,NaN,NaN,639.65,399,0.0,130.32,0.115182,0.098841,40.606701,f,37.91,15.53,15.53,1,74.59,5,lxidpiddsbxsbosboudacockeimpuepw,17.321,0.298927,81.131934,0.099642,27.043978,-0.192682,0.401613,1.903597,0.231871,-96.740136,173.205081,1.0,1.0,1.0,1.0,1.0,1.0,271.410525
3,cf81de72ff7997ed10729751059cf7a3,apdekpcbwosbxepsfxclislboipuxpop,foosdfpfkusacimwkcsosbicdxkicaua,186838,0,12091,03/01/2017,03/01/2023,03/01/2017,04/01/2022,188.576187,188.576187,NaN,NaN,2634.23,738,0.0,16.42,0.164058,0.086163,44.311378,f,93.12,27.00,27.00,1,210.18,5,lxidpiddsbxsbosboudacockeimpuepw,11.951,0.257093,44.266930,0.085698,14.755643,0.991480,0.000000,2.181068,0.000000,-396.004507,0.000000,1.0,1.0,0.0,1.0,0.0,0.0,172.182556
4,c7c8ee9daf2a864d159d45a2373ada72,NaN,usilxuppasemubllopkaafesmlibmsdf,19051,0,0,07/10/2017,07/10/2022,27/08/2021,09/10/2021,68.635785,68.635785,NaN,NaN,2801.47,0,0.0,0.00,0.141434,0.000000,44.311378,f,0.00,8.62,8.62,1,223.23,4,lxidpiddsbxsbosboudacockeimpuepw,13.150,0.144582,44.355821,0.048194,14.785274,-0.391589,0.401609,1.249749,0.200805,-112.315574,200.001800,1.0,0.0,0.0,1.0,0.0,0.0,306.787663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,18463073fb097fc0ac5d3e040f356987,NaN,foosdfpfkusacimwkcsosbicdxkicaua,32270,47940,0,24/05/2018,08/05/2022,08/05/2021,26/05/2020,68.635785,68.635785,NaN,NaN,4648.01,0,0.0,18.57,0.138305,0.000000,44.311378,t,0.00,27.88,27.88,2,381.77,4,lxidpiddsbxsbosboudacockeimpuepw,15.000,0.144349,44.444710,0.048116,14.814903,0.394961,0.401609,2.042683,0.231870,-104.068875,173.206834,1.0,0.0,0.0,1.0,0.0,0.0,307.896529
20116,d0a6f71671571ed83b2645d23af6de00,NaN,foosdfpfkusacimwkcsosbicdxkicaua,7223,0,181,27/08/2018,27/08/2022,27/08/2018,28/08/2021,68.640000,68.640000,1254.65,15.94,631.69,181,0.0,144.03,0.100167,0.091892,58.995952,f,15.94,0.00,0.00,1,90.34,3,lxidpiddsbxsbosboudacockeimpuepw,6.000,0.272930,103.615764,0.090977,34.538588,-0.727070,0.401603,1.757212,0.283976,-89.216410,141.421356,1.0,1.0,1.0,1.0,1.0,1.0,379.642267
20117,10e6828ddd62cbcf687cb74928c4c2d2,NaN,foosdfpfkusacimwkcsosbicdxkicaua,1844,0,179,08/02/2018,07/02/2022,08/02/2018,09/02/2021,91.884542,91.884542,NaN,NaN,190.39,179,0.0,129.60,0.116900,0.100015

<b>Fazendo ajustes na coluna de has_gas.</b>

O modelo não entende "t" e "f", por conta disso, estamos mudando para True e False. Depois disso é necessário mudar o tipo do dado da coluna para int (inteiro), para que o modelo entenda que a informação é binária.

In [27]:
pc['has_gas'] = pc['has_gas'].map({'t': True, 'f': False})
pc['has_gas'] = pc['has_gas'].astype(int)
pc

,id,activity_new,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_base_bill_ele,forecast_base_bill_year,forecast_bill_12m,forecast_cons,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_p1,forecast_price_energy_p2,forecast_price_pow_p1,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,total_var,total_fix,total_var_mean,total_fix_mean,highest_variation_var,highest_variation_fix,std_variation_var,std_variation_fix,cv_variation_var,cv_variation_fix,p1_var,p2_var,p3_var,p1_fix,p2_fix,p3_fix,kwh_consumed
0,ed18c9981ce183ef347246cdcc55f743,bwpaswkpcilmlklklcapcwwumwaodaoo,foosdfpfkusacimwkcsosbicdxkicaua,42132,0,3215,06/09/2017,06/09/2022,27/08/2021,07/09/2021,451.048858,451.048858,NaN,NaN,4001.08,3215,0.0,130.43,0.114603,0.098261,40.606701,0,296.90,9.63,9.63,1,488.25,4,lxidpiddsbxsbosboudacockeimpuepw,20.000,0.293558,81.457770,0.097853,27.152590,0.596262,0.000000,2.082828,0.000000,-119.379663,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,277.484415
1,9c341e73685fcff6e3527ebd51da3fe5,NaN,foosdfpfkusacimwkcsosbicdxkicaua,3537,0,563,13/12/2016,31/12/2022,13/12/2016,01/01/2022,172.527461,172.527461,NaN,NaN,518.22,563,0.0,23.20,0.142174,0.000000,44.444710,0,80.66,0.12,0.12,1,29.18,5,lxidpiddsbxsbosboudacockeimpuepw,15.000,0.143620,44.444710,0.047873,14.814903,-3.013195,0.000000,0.000000,0.000000,-0.000000,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,309.460451
2,97dd64b33716aadc4e22c4ab85d1c7a4,NaN,foosdfpfkusacimwkcsosbicdxkicaua,6543,0,399,04/09/2016,04/09/2022,27/08/2021,05/09/2021,117.464615,117.464615,NaN,NaN,639.65,399,0.0,130.32,0.115182,0.098841,40.606701,0,37.91,15.53,15.53,1,74.59,5,lxidpiddsbxsbosboudacockeimpuepw,17.321,0.298927,81.131934,0.099642,27.043978,-0.192682,0.401613,1.903597,0.231871,-96.740136,173.205081,1.0,1.0,1.0,1.0,1.0,1.0,271.410525
3,cf81de72ff7997ed10729751059cf7a3,apdekpcbwosbxepsfxclislboipuxpop,foosdfpfkusacimwkcsosbicdxkicaua,186838,0,12091,03/01/2017,03/01/2023,03/01/2017,04/01/2022,188.576187,188.576187,NaN,NaN,2634.23,738,0.0,16.42,0.164058,0.086163,44.311378,0,93.12,27.00,27.00,1,210.18,5,lxidpiddsbxsbosboudacockeimpuepw,11.951,0.257093,44.266930,0.085698,14.755643,0.991480,0.000000,2.181068,0.000000,-396.004507,0.000000,1.0,1.0,0.0,1.0,0.0,0.0,172.182556
4,c7c8ee9daf2a864d159d45a2373ada72,NaN,usilxuppasemubllopkaafesmlibmsdf,19051,0,0,07/10/2017,07/10/2022,27/08/2021,09/10/2021,68.635785,68.635785,NaN,NaN,2801.47,0,0.0,0.00,0.141434,0.000000,44.311378,0,0.00,8.62,8.62,1,223.23,4,lxidpiddsbxsbosboudacockeimpuepw,13.150,0.144582,44.355821,0.048194,14.785274,-0.391589,0.401609,1.249749,0.200805,-112.315574,200.001800,1.0,0.0,0.0,1.0,0.0,0.0,306.787663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,18463073fb097fc0ac5d3e040f356987,NaN,foosdfpfkusacimwkcsosbicdxkicaua,32270,47940,0,24/05/2018,08/05/2022,08/05/2021,26/05/2020,68.635785,68.635785,NaN,NaN,4648.01,0,0.0,18.57,0.138305,0.000000,44.311378,1,0.00,27.88,27.88,2,381.77,4,lxidpiddsbxsbosboudacockeimpuepw,15.000,0.144349,44.444710,0.048116,14.814903,0.394961,0.401609,2.042683,0.231870,-104.068875,173.206834,1.0,0.0,0.0,1.0,0.0,0.0,307.896529
20116,d0a6f71671571ed83b2645d23af6de00,NaN,foosdfpfkusacimwkcsosbicdxkicaua,7223,0,181,27/08/2018,27/08/2022,27/08/2018,28/08/2021,68.640000,68.640000,1254.65,15.94,631.69,181,0.0,144.03,0.100167,0.091892,58.995952,0,15.94,0.00,0.00,1,90.34,3,lxidpiddsbxsbosboudacockeimpuepw,6.000,0.272930,103.615764,0.090977,34.538588,-0.727070,0.401603,1.757212,0.283976,-89.216410,141.421356,1.0,1.0,1.0,1.0,1.0,1.0,379.642267
20117,10e6828ddd62cbcf687cb74928c4c2d2,NaN,foosdfpfkusacimwkcsosbicdxkicaua,1844,0,179,08/02/2018,07/02/2022,08/02/2018,09/02/2021,91.884542,91.884542,NaN,NaN,190.39,179,0.0,129.60,0.116900,0.100015

<b>Limpeza primeira camada</b>

O código abaixo faz uma limpeza no dataframe. Não houve nenhuma linha ou coluna excluída, mas podemos ver que reduziu espaço de memória. Fizemos isso para conseguirmos trabalhar melhor com o dataframe.

In [28]:
# cleaning data with a libraty called klib
pc_cleaned = klib.data_cleaning(pc)
pc_cleaned

Shape of cleaned data: (20120, 47) - Remaining NAs: 49685


Dropped rows: 0
     of which 0 duplicates. (Rows (first 150 shown): [])

Dropped columns: 0
     of which 0 single valued.     Columns: []
Dropped missing values: 0
Reduced memory by at least: 3.06 MB (-42.86%)



,id,activity_new,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_base_bill_ele,forecast_base_bill_year,forecast_bill_12m,forecast_cons,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_p1,forecast_price_energy_p2,forecast_price_pow_p1,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,total_var,total_fix,total_var_mean,total_fix_mean,highest_variation_var,highest_variation_fix,std_variation_var,std_variation_fix,cv_variation_var,cv_variation_fix,p1_var,p2_var,p3_var,p1_fix,p2_fix,p3_fix,kwh_consumed
0,ed18c9981ce183ef347246cdcc55f743,bwpaswkpcilmlklklcapcwwumwaodaoo,foosdfpfkusacimwkcsosbicdxkicaua,42132,0,3215,06/09/2017,06/09/2022,27/08/2021,07/09/2021,451.048858,451.048858,NaN,NaN,4001.08,3215,0.0,130.429993,0.114603,0.098261,40.606701,0,296.899994,9.630000,9.630000,1,488.250000,4,lxidpiddsbxsbosboudacockeimpuepw,20.000000,0.293558,81.457771,0.097853,27.152590,0.596262,0.000000,2.082828,0.000000,-119.379663,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,277.484406
1,9c341e73685fcff6e3527ebd51da3fe5,NaN,foosdfpfkusacimwkcsosbicdxkicaua,3537,0,563,13/12/2016,31/12/2022,13/12/2016,01/01/2022,172.527461,172.527461,NaN,NaN,518.22,563,0.0,23.200001,0.142174,0.000000,44.444710,0,80.660004,0.120000,0.120000,1,29.180000,5,lxidpiddsbxsbosboudacockeimpuepw,15.000000,0.143620,44.444710,0.047873,14.814903,-3.013195,0.000000,0.000000,0.000000,-0.000000,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,309.460449
2,97dd64b33716aadc4e22c4ab85d1c7a4,NaN,foosdfpfkusacimwkcsosbicdxkicaua,6543,0,399,04/09/2016,04/09/2022,27/08/2021,05/09/2021,117.464615,117.464615,NaN,NaN,639.65,399,0.0,130.320007,0.115182,0.098841,40.606701,0,37.910000,15.530000,15.530000,1,74.589996,5,lxidpiddsbxsbosboudacockeimpuepw,17.320999,0.298927,81.131935,0.099642,27.043978,-0.192682,0.401613,1.903597,0.231871,-96.740136,173.205081,1.0,1.0,1.0,1.0,1.0,1.0,271.410522
3,cf81de72ff7997ed10729751059cf7a3,apdekpcbwosbxepsfxclislboipuxpop,foosdfpfkusacimwkcsosbicdxkicaua,186838,0,12091,03/01/2017,03/01/2023,03/01/2017,04/01/2022,188.576187,188.576187,NaN,NaN,2634.23,738,0.0,16.420000,0.164058,0.086163,44.311378,0,93.120003,27.000000,27.000000,1,210.179993,5,lxidpiddsbxsbosboudacockeimpuepw,11.951000,0.257093,44.266930,0.085698,14.755643,0.991480,0.000000,2.181068,0.000000,-396.004507,0.000000,1.0,1.0,0.0,1.0,0.0,0.0,172.182556
4,c7c8ee9daf2a864d159d45a2373ada72,NaN,usilxuppasemubllopkaafesmlibmsdf,19051,0,0,07/10/2017,07/10/2022,27/08/2021,09/10/2021,68.635785,68.635785,NaN,NaN,2801.47,0,0.0,0.000000,0.141434,0.000000,44.311378,0,0.000000,8.620000,8.620000,1,223.229996,4,lxidpiddsbxsbosboudacockeimpuepw,13.150000,0.144581,44.355820,0.048194,14.785274,-0.391589,0.401609,1.249749,0.200805,-112.315574,200.001800,1.0,0.0,0.0,1.0,0.0,0.0,306.787659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,18463073fb097fc0ac5d3e040f356987,NaN,foosdfpfkusacimwkcsosbicdxkicaua,32270,47940,0,24/05/2018,08/05/2022,08/05/2021,26/05/2020,68.635785,68.635785,NaN,NaN,4648.01,0,0.0,18.570000,0.138305,0.000000,44.311378,1,0.000000,27.879999,27.879999,2,381.769989,4,lxidpiddsbxsbosboudacockeimpuepw,15.000000,0.144350,44.444710,0.048117,14.814903,0.394961,0.401609,2.042683,0.231870,-104.068875,173.206834,1.0,0.0,0.0,1.0,0.0,0.0,307.896515
20116,d0a6f71671571ed83b2645d23af6de00,NaN,foosdfpfkusacimwkcsosbicdxkicaua,7223,0,181,27/08/2018,27/08/2022,27/08/2018,28/08/2021,68.640000,68.640000,1254.65,15.94,631.69,181,0.0,144.029999,0.100167,0.091892,58.995953,0,15.940000,0.000000,0.000000,1,90.339996,3,lxidpiddsbxsbosboudacockeimpuepw,6.000000,0.272930,103.615761,0.090977,34.538589,-0.727070,0.401603,1.757212,0.283976,-89.216410,141.421356,1.0,1.0,1.0,1.0,1.0,1.0,379.642273
20117,10e6828ddd62cbcf687cb74928c4c2d

<b>Tratamento de missing values</b>

Na primeira parte, está sendo inserido o valor "unknown" nos campos com valores nulos da coluna de channe_sales. A mesma ação está sendo tomada com a coluna origin_up. Na última parte, estamos substituindo os campos nulos na coluna de activity_new por 0 e os campos com valores para 1. 

A razão para substituir valores nulos por "unknow" é que consideramos a falta de dados nas colunas como correta, ou seja, não temos dados do canal de vendas porque não houve um canal. 

Já no caso da substituição de valores na coluna de activity_new está relacionada a uma correlação observada, onde sempre que há dados os valores são altos e quando não há são baixos. A hipótese adotada é a de que a presença de dados significa que o cliente é uma empresa, e quando não há seria uma residência ou algo do tipo.

In [29]:
# Changing the nan values to unknown
pc_cleaned['channel_sales'] = pc_cleaned['channel_sales'].replace(np.nan, 'unknown')
pc_encoded = pd.get_dummies(pc_cleaned, columns=['channel_sales'])

# Changing the nan values to unknown
pc_encoded['origin_up'] = pc_encoded['origin_up'].replace(np.nan, 'unknown')
pc_encoded = pd.get_dummies(pc_encoded, columns=['origin_up'])

# 1 if the line is not null and 0 otherwise
pc_encoded['activity_new'] = pc_encoded['activity_new'].notna().astype(int)
pc_encoded.rename(columns={'activity_new': 'is_company'}, inplace=True)
pc_encoded

,id,is_company,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_base_bill_ele,forecast_base_bill_year,forecast_bill_12m,forecast_cons,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_p1,forecast_price_energy_p2,forecast_price_pow_p1,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,pow_max,total_var,total_fix,total_var_mean,total_fix_mean,highest_variation_var,highest_variation_fix,std_variation_var,std_variation_fix,cv_variation_var,cv_variation_fix,p1_var,p2_var,p3_var,p1_fix,p2_fix,p3_fix,kwh_consumed,channel_sales_epumfxlbckeskwekxbiuasklxalciiuu,channel_sales_ewpakwlliwisiwduibdlfmalxowmwpci,channel_sales_fixdbufsefwooaasfcxdxadsiekoceaa,channel_sales_foosdfpfkusacimwkcsosbicdxkicaua,channel_sales_lmkebamcaaclubfxadlmueccxoimlema,channel_sales_sddiedcslfslkckwlfkdpoeeailfpeds,channel_sales_usilxuppasemubllopkaafesmlibmsdf,origin_up_aabpopmuoobccoxasfsksebxoxffdcxs,origin_up_ewxeelcelemmiwuafmddpobolfuxioce,origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_up_ldkssxwpmemidmecebumciepifcamkci,origin_up_lxidpiddsbxsbosboudacockeimpuepw,origin_up_usapbepcfoloekilkwsdiboslwaxobdp
0,ed18c9981ce183ef347246cdcc55f743,1,42132,0,3215,06/09/2017,06/09/2022,27/08/2021,07/09/2021,451.048858,451.048858,NaN,NaN,4001.08,3215,0.0,130.429993,0.114603,0.098261,40.606701,0,296.899994,9.630000,9.630000,1,488.250000,4,20.000000,0.293558,81.457771,0.097853,27.152590,0.596262,0.000000,2.082828,0.000000,-119.379663,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,277.484406,False,False,False,True,False,False,False,False,False,False,False,True,False
1,9c341e73685fcff6e3527ebd51da3fe5,0,3537,0,563,13/12/2016,31/12/2022,13/12/2016,01/01/2022,172.527461,172.527461,NaN,NaN,518.22,563,0.0,23.200001,0.142174,0.000000,44.444710,0,80.660004,0.120000,0.120000,1,29.180000,5,15.000000,0.143620,44.444710,0.047873,14.814903,-3.013195,0.000000,0.000000,0.000000,-0.000000,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,309.460449,False,False,False,True,False,False,False,False,False,False,False,True,False
2,97dd64b33716aadc4e22c4ab85d1c7a4,0,6543,0,399,04/09/2016,04/09/2022,27/08/2021,05/09/2021,117.464615,117.464615,NaN,NaN,639.65,399,0.0,130.320007,0.115182,0.098841,40.606701,0,37.910000,15.530000,15.530000,1,74.589996,5,17.320999,0.298927,81.131935,0.099642,27.043978,-0.192682,0.401613,1.903597,0.231871,-96.740136,173.205081,1.0,1.0,1.0,1.0,1.0,1.0,271.410522,False,False,False,True,False,False,False,False,False,False,False,True,False
3,cf81de72ff7997ed10729751059cf7a3,1,186838,0,12091,03/01/2017,03/01/2023,03/01/2017,04/01/2022,188.576187,188.576187,NaN,NaN,2634.23,738,0.0,16.420000,0.164058,0.086163,44.311378,0,93.120003,27.000000,27.000000,1,210.179993,5,11.951000,0.257093,44.266930,0.085698,14.755643,0.991480,0.000000,2.181068,0.000000,-396.004507,0.000000,1.0,1.0,0.0,1.0,0.0,0.0,172.182556,False,False,False,True,False,False,False,False,False,False,False,True,False
4,c7c8ee9daf2a864d159d45a2373ada72,0,19051,0,0,07/10/2017,07/10/2022,27/08/2021,09/10/2021,68.635785,68.635785,NaN,NaN,2801.47,0,0.0,0.000000,0.141434,0.000000,44.311378,0,0.000000,8.620000,8.620000,1,223.229996,4,13.150000,0.144581,44.355820,0.048194,14.785274,-0.391589,0.401609,1.249749,0.200805,-112.315574,200.001800,1.0,0.0,0.0,1.0,0.0,0.0,306.787659,False,False,False,False,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,18463073fb097fc0ac5d3e040f356987,0,32270,47940,0,24/05/2018,08/05/2022,08/05/2021,26/05/2020,68.635785,68.635785,NaN,NaN,4648.01,0,0.0,18.570000,0.138305,0.000000,44.311378,1,0.000000,27.879999,27.879999,2,381.769989,4,15.000000,0.144350,44.444710,0.048117,14.814903,0.394961,0.401609,2.042683,0.231870,-104.068875,17

<b>Eliminação de colunas</b>

Foi identificado pouca relevância das colunas de "forecast_base_bil_year", "forecast_bill_12m", "forecast_cons" porque existem muitos poucos dados e a sua correlação com outras colunas é tão alta que poderia ser considerado valores duplicados e que podem inviesar o modelo. Por isso as colunas foram excluídas.

In [30]:
# droping columns with missing values
pc = pc_encoded.drop(['forecast_base_bill_year', 'forecast_bill_12m', 'forecast_cons'], axis=1)
pc = pc.dropna()
pc

,id,is_company,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_base_bill_ele,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_p1,forecast_price_energy_p2,forecast_price_pow_p1,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,pow_max,total_var,total_fix,total_var_mean,total_fix_mean,highest_variation_var,highest_variation_fix,std_variation_var,std_variation_fix,cv_variation_var,cv_variation_fix,p1_var,p2_var,p3_var,p1_fix,p2_fix,p3_fix,kwh_consumed,channel_sales_epumfxlbckeskwekxbiuasklxalciiuu,channel_sales_ewpakwlliwisiwduibdlfmalxowmwpci,channel_sales_fixdbufsefwooaasfcxdxadsiekoceaa,channel_sales_foosdfpfkusacimwkcsosbicdxkicaua,channel_sales_lmkebamcaaclubfxadlmueccxoimlema,channel_sales_sddiedcslfslkckwlfkdpoeeailfpeds,channel_sales_usilxuppasemubllopkaafesmlibmsdf,origin_up_aabpopmuoobccoxasfsksebxoxffdcxs,origin_up_ewxeelcelemmiwuafmddpobolfuxioce,origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_up_ldkssxwpmemidmecebumciepifcamkci,origin_up_lxidpiddsbxsbosboudacockeimpuepw,origin_up_usapbepcfoloekilkwsdiboslwaxobdp
0,ed18c9981ce183ef347246cdcc55f743,1,42132,0,3215,06/09/2017,06/09/2022,27/08/2021,07/09/2021,451.048858,4001.08,3215,0.0,130.429993,0.114603,0.098261,40.606701,0,296.899994,9.630000,9.630000,1,488.250000,4,20.000000,0.293558,81.457771,0.097853,27.152590,0.596262,0.000000,2.082828,0.000000,-119.379663,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,277.484406,False,False,False,True,False,False,False,False,False,False,False,True,False
1,9c341e73685fcff6e3527ebd51da3fe5,0,3537,0,563,13/12/2016,31/12/2022,13/12/2016,01/01/2022,172.527461,518.22,563,0.0,23.200001,0.142174,0.000000,44.444710,0,80.660004,0.120000,0.120000,1,29.180000,5,15.000000,0.143620,44.444710,0.047873,14.814903,-3.013195,0.000000,0.000000,0.000000,-0.000000,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,309.460449,False,False,False,True,False,False,False,False,False,False,False,True,False
2,97dd64b33716aadc4e22c4ab85d1c7a4,0,6543,0,399,04/09/2016,04/09/2022,27/08/2021,05/09/2021,117.464615,639.65,399,0.0,130.320007,0.115182,0.098841,40.606701,0,37.910000,15.530000,15.530000,1,74.589996,5,17.320999,0.298927,81.131935,0.099642,27.043978,-0.192682,0.401613,1.903597,0.231871,-96.740136,173.205081,1.0,1.0,1.0,1.0,1.0,1.0,271.410522,False,False,False,True,False,False,False,False,False,False,False,True,False
3,cf81de72ff7997ed10729751059cf7a3,1,186838,0,12091,03/01/2017,03/01/2023,03/01/2017,04/01/2022,188.576187,2634.23,738,0.0,16.420000,0.164058,0.086163,44.311378,0,93.120003,27.000000,27.000000,1,210.179993,5,11.951000,0.257093,44.266930,0.085698,14.755643,0.991480,0.000000,2.181068,0.000000,-396.004507,0.000000,1.0,1.0,0.0,1.0,0.0,0.0,172.182556,False,False,False,True,False,False,False,False,False,False,False,True,False
4,c7c8ee9daf2a864d159d45a2373ada72,0,19051,0,0,07/10/2017,07/10/2022,27/08/2021,09/10/2021,68.635785,2801.47,0,0.0,0.000000,0.141434,0.000000,44.311378,0,0.000000,8.620000,8.620000,1,223.229996,4,13.150000,0.144581,44.355820,0.048194,14.785274,-0.391589,0.401609,1.249749,0.200805,-112.315574,200.001800,1.0,0.0,0.0,1.0,0.0,0.0,306.787659,False,False,False,False,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,18463073fb097fc0ac5d3e040f356987,0,32270,47940,0,24/05/2018,08/05/2022,08/05/2021,26/05/2020,68.635785,4648.01,0,0.0,18.570000,0.138305,0.000000,44.311378,1,0.000000,27.879999,27.879999,2,381.769989,4,15.000000,0.144350,44.444710,0.048117,14.814903,0.394961,0.401609,2.042683,0.231870,-104.068875,173.206834,1.0,0.0,0.0,1.0,0.0,0.0,307.896515,False,False,False,True,False,False,False,False,False,False,False,True,False
20116,d0a6f71671571ed83b2645d23af6de00,0,7223,0,181,27/08/20

<b>Valores inválidos</b>


Existem datas inválidas no banco de dados. Ex: 29/02/2018, OBS: 2018 não é ano bissexto. O código itera sobre as features especificadas e quando encontra uma data inválida a substitui por outra data correspondente. Depois cria-se uma coluna de ano ao invés da data completa, foi entendido que dia a mês não são tão relevantes e que essa informação seria melhor aproveitada dessa maneira.

In [31]:
date_cols = ['date_activ', 'date_end', 'date_modif_prod', 'date_renewal'] # Date columns
invalid_dates = {"29/02/2018": "28/02/2018", "29/02/2022": "28/02/2022"} # Invalid dates

# Replace invalid dates with the most likely correct date
for col in date_cols:
    for inv_date, rep_date in invalid_dates.items():
        pc[col] = pc[col].replace(inv_date, rep_date)
for col in date_cols:
    pc[col + '_year'] = pd.to_datetime(pc[col], dayfirst=True).dt.year

# Droping the original date columns 
pc = pc.drop(columns=date_cols)
pc

,id,is_company,cons_12m,cons_gas_12m,cons_last_month,forecast_base_bill_ele,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_p1,forecast_price_energy_p2,forecast_price_pow_p1,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,pow_max,total_var,total_fix,total_var_mean,total_fix_mean,highest_variation_var,highest_variation_fix,std_variation_var,std_variation_fix,cv_variation_var,cv_variation_fix,p1_var,p2_var,p3_var,p1_fix,p2_fix,p3_fix,kwh_consumed,channel_sales_epumfxlbckeskwekxbiuasklxalciiuu,channel_sales_ewpakwlliwisiwduibdlfmalxowmwpci,channel_sales_fixdbufsefwooaasfcxdxadsiekoceaa,channel_sales_foosdfpfkusacimwkcsosbicdxkicaua,channel_sales_lmkebamcaaclubfxadlmueccxoimlema,channel_sales_sddiedcslfslkckwlfkdpoeeailfpeds,channel_sales_usilxuppasemubllopkaafesmlibmsdf,origin_up_aabpopmuoobccoxasfsksebxoxffdcxs,origin_up_ewxeelcelemmiwuafmddpobolfuxioce,origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_up_ldkssxwpmemidmecebumciepifcamkci,origin_up_lxidpiddsbxsbosboudacockeimpuepw,origin_up_usapbepcfoloekilkwsdiboslwaxobdp,date_activ_year,date_end_year,date_modif_prod_year,date_renewal_year
0,ed18c9981ce183ef347246cdcc55f743,1,42132,0,3215,451.048858,4001.08,3215,0.0,130.429993,0.114603,0.098261,40.606701,0,296.899994,9.630000,9.630000,1,488.250000,4,20.000000,0.293558,81.457771,0.097853,27.152590,0.596262,0.000000,2.082828,0.000000,-119.379663,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,277.484406,False,False,False,True,False,False,False,False,False,False,False,True,False,2017,2022,2021,2021
1,9c341e73685fcff6e3527ebd51da3fe5,0,3537,0,563,172.527461,518.22,563,0.0,23.200001,0.142174,0.000000,44.444710,0,80.660004,0.120000,0.120000,1,29.180000,5,15.000000,0.143620,44.444710,0.047873,14.814903,-3.013195,0.000000,0.000000,0.000000,-0.000000,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,309.460449,False,False,False,True,False,False,False,False,False,False,False,True,False,2016,2022,2016,2022
2,97dd64b33716aadc4e22c4ab85d1c7a4,0,6543,0,399,117.464615,639.65,399,0.0,130.320007,0.115182,0.098841,40.606701,0,37.910000,15.530000,15.530000,1,74.589996,5,17.320999,0.298927,81.131935,0.099642,27.043978,-0.192682,0.401613,1.903597,0.231871,-96.740136,173.205081,1.0,1.0,1.0,1.0,1.0,1.0,271.410522,False,False,False,True,False,False,False,False,False,False,False,True,False,2016,2022,2021,2021
3,cf81de72ff7997ed10729751059cf7a3,1,186838,0,12091,188.576187,2634.23,738,0.0,16.420000,0.164058,0.086163,44.311378,0,93.120003,27.000000,27.000000,1,210.179993,5,11.951000,0.257093,44.266930,0.085698,14.755643,0.991480,0.000000,2.181068,0.000000,-396.004507,0.000000,1.0,1.0,0.0,1.0,0.0,0.0,172.182556,False,False,False,True,False,False,False,False,False,False,False,True,False,2017,2023,2017,2022
4,c7c8ee9daf2a864d159d45a2373ada72,0,19051,0,0,68.635785,2801.47,0,0.0,0.000000,0.141434,0.000000,44.311378,0,0.000000,8.620000,8.620000,1,223.229996,4,13.150000,0.144581,44.355820,0.048194,14.785274,-0.391589,0.401609,1.249749,0.200805,-112.315574,200.001800,1.0,0.0,0.0,1.0,0.0,0.0,306.787659,False,False,False,False,False,False,True,False,False,False,False,True,False,2017,2022,2021,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,18463073fb097fc0ac5d3e040f356987,0,32270,47940,0,68.635785,4648.01,0,0.0,18.570000,0.138305,0.000000,44.311378,1,0.000000,27.879999,27.879999,2,381.769989,4,15.000000,0.144350,44.444710,0.048117,14.814903,0.394961,0.401609,2.042683,0.231870,-104.068875,173.206834,1.0,0.0,0.0,1.0,0.0,0.0,307.896515,False,False,False,True,False,False,False,False,False,False,False,True,False,2018,2022,2021,2020
20116,d0a6f71671571ed83b2645d23af6de00,0,7223,0,181,68.640000,631.69,181,0.0,144.029999,0.100167,0.091892,58.995953,0,15.940000,0.000000,0.000000,1,90.339996,3,6.000000,0.272930,103.61

<b>Remoção de valores negativos</b>

Verificamos que existem valores negativos em algumas colunas, então excluimos esses valores negativos porque entendemos que eles não podem ser usados pois podem inviesar o modelo.

In [32]:
# Removing negative values from the dataset
columns_to_remove_negatives = [
    'cons_12m', 'cons_gas_12m', 'cons_last_month', 'forecast_cons_12m',
    'forecast_cons_year', 'forecast_meter_rent_12m', 'forecast_price_pow_p1',
    'imp_cons', 'net_margin'
]
for col in columns_to_remove_negatives:
    pc = pc[pc[col] >= 0]

<b>Remoção de valores negativos</b>

Foi encontrado valores negativos nas colunas de margem líquida (margin_net_pow_ele) e margem bruta (margin_gross_pow_ele). Esses valores precisam ser ajustados. Entendemos que como essa margem não pode ser negativa deve ter sido um erro de digitação (erro de input), por isso resolvemos substituir os valores negativos pelos valores absolutos. Foi escolhida esta abordagem ao invés da substituição por valores estatísticos porque acreditamos que dessa maneira a distribuição real dos dados será preservada.

In [33]:
# Taking absolute values of negative values in the margin columns
pc['margin_net_pow_ele'] = pc['margin_net_pow_ele'].abs()
pc['margin_gross_pow_ele'] = pc['margin_gross_pow_ele'].abs()

<b>Remoção de valores negativos</b>

Identificamos valores negativos nas colunas de consumo dos últimos 12 meses (cons_12m). Por isso escolhemos substituir os valores errados pela média da coluna.

In [34]:
# Taking the median of the cons_12m column and replacing the negative values with it
median_positive = pc.loc[pc['cons_12m'] > 0, 'cons_12m'].median()
pc['cons_12m_corrected'] = pc['cons_12m'].apply(lambda x: median_positive if x < 0 else x)

# Taking the log of the corrected column to reduce the skewness and help the model
pc['cons_12m_log'] = np.log1p(pc['cons_12m_corrected'])

# Dropping the original column
pc = pc.drop('cons_12m', axis=1)

In [35]:
pc.shape

(19453, 56)

<b>Identificação de valores infinitos nos dados</b>

O grupo identificou a necessidade de verificar a presença de valores infinitos nas colunas numéricas do dataframe, uma vez que estes valores podem atrapalhar na análise e modelagem dos dados. A função check_infinite_values foi criada para identificar esses valores e retorná-los para futura correção ou exclusão.

In [36]:
def check_infinite_values(df):
    infinite_values = {}
    # Select only numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        infinite_indexes = df.index[np.isinf(df[col])]
        if not infinite_indexes.empty:
            infinite_values[col] = df.loc[infinite_indexes, col].tolist()
    return infinite_values

infinite_values = check_infinite_values(pc)
if infinite_values:
    print("Infinite values found:")
    for col, values in infinite_values.items():
        print(f"Column: {col} - Values: {values}")
        print(len(values))
else:
    print("No infinite values found in the dataset.")


Infinite values found:
Column: highest_variation_var - Values: [inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
60
Column: highest_variation_fix - Values: [inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
39


<b>Removendo linhas com valores infinitos</b>

Para garantir a qualidade e consistência dos dados, o grupo identificou a necessidade de excluir as linhas que contêm valores infinitos nas colunas numéricas. Estes valores podem ser resultantes de operações de divisão por zero ou outras operações que produzam valores infinitos e, portanto, podem prejudicar a análise e modelagem dos dados. A função drop_infinite_values foi criada para realizar essa tarefa e retornar um dataframe sem as linhas contendo valores infinitos.

In [37]:
def drop_infinite_values(df):
    numeric_data = df.select_dtypes(include=[np.number])

    infinite_indexes = df.index[np.isinf(numeric_data).any(axis=1)]
    
    df_dropped_inf = df.drop(index=infinite_indexes)
    
    return df_dropped_inf

pc = drop_infinite_values(pc)

# 2 Modelagem e Validação

### 2.1 Dividindo os dados para treinamento, validação cruzada e teste

<b>Unindo, dividindo e verificando os dados para modelagem</b>

O grupo percebeu a necessidade de combinar diferentes conjuntos de dados, isolando as características (features) e os alvos (target), e dividindo os dados em conjuntos de treinamento e teste. Este processo é fundamental para a posterior construção e avaliação do modelo.

Os dados são então divididos em conjuntos de treinamento e teste, usando 80% dos dados para treinamento e os 20% restantes para teste. Isso permite que o modelo seja treinado e avaliado em dados separados, garantindo uma avaliação justa de sua performance.

Para garantir que a divisão foi feita corretamente e que ambos os conjuntos (treinamento e teste) têm uma distribuição similar da variável alvo, imprimimos a contagem de valores.

In [38]:
# Merge the data
train_val_data = pd.merge(pc, hc, on='id', how='inner')

# Split the data into features and target
X = train_val_data.drop(columns=['id', 'churn'])
y = train_val_data['churn']

# Split the data into training and test sets (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
# Initialize the scaler
scaler = RobustScaler()

# Fit on the training data and transform both training and test data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  # Use transform, not fit_transform on test data

In [42]:
# PCA
from sklearn.decomposition import PCA

# Let's reduce the data to 20 components using PCA
pca = PCA(n_components=20)

# Fit on the training data and transform both training and test data
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)  # Use transform, not fit_transform on test data

##### XGBClassifier

<b>Classificação usando XGBoost</b>

Utilizamos o XGBClassifier para treinar um modelo de classificação. Os hiperparâmetros foram escolhidos com base em experimentos anteriores ou otimizações:

learning_rate: 0.052476590847462534<br>
max_depth: 10<br>
min_child_weight: 0<br>
n_estimators: 171<br>
n_jobs: 1 (Usa um único núcleo)<br>
subsample: 0.8688491794028068<br>
verbosity: 1 (Mostra mensagens de saída)<br>
scale_pos_weight: Equilibra as classes positivas e negativas, com base no conjunto de treinamento.<br>

<b>Métricas de Avaliação</b><br>

Estamos utilizando diversas métricas para avaliar o desempenho do modelo:<br>

Acurácia<br>
Precisão (média macro)<br>
Recall (média macro)<br>
F1 (média macro)<br>
MSE (Mean Squared Error)<br>
ROC AUC<br>

<b>Validação Cruzada</b><br>

Para avaliar de maneira mais robusta, utilizamos a técnica de validação cruzada estratificada (StratifiedKFold). Escolhemos fazer 20 divisões (folds) e, para garantir a reprodutibilidade dos resultados, definimos uma semente (random_state=42).<br>

<b>Resultados da Avaliação</b><br>

Após realizar a validação cruzada com o modelo e os dados fornecidos, exibimos a média das métricas de avaliação para entender o desempenho geral do modelo.<br>

In [ ]:
# Set the parameters
params = {
  'objective': 'binary:logistic',
  'verbosity': 0,
  'booster': 'gbtree',
  'tree_method': 'gpu_hist',
  'learning_rate': 0.005171676107542148,
  'n_estimators': 593,
  'max_depth': 15,
  'subsample': 0.8068033937233334,
  'colsample_bytree': 0.9680836794118163,
}

model = xgb.XGBClassifier(**params)

scoring = {
    'precision': make_scorer(precision_score, average='macro'),
    'recall': make_scorer(recall_score, average='macro'),
    'f1': make_scorer(f1_score, average='macro'),
    'auc-roc': 'roc_auc',
    'accuracy': 'accuracy'
}

stratified_kfold = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)

cv_results = cross_validate(model, X_train_pca, y_train, cv=stratified_kfold, scoring=scoring)

for metric, scores in cv_results.items():
    if metric.startswith('test_'):
        print(f"{metric[5:]}: {scores.mean() * 100:.2f}%")

<b>Aplicação do XGBClassifier</b>

O modelo XGBoost, que foi configurado anteriormente, é treinado usando o conjunto de dados de treino X_train_val e y_train_val.

Posteriormente, fazemos predições usando o conjunto de testes X_test. O modelo não apenas prevê as classes diretamente, mas também fornece probabilidades associadas a cada classe.

<b>Calculando as métricas de desempenho no conjunto de testes:</b>

Com as predições em mãos, avaliamos o desempenho do modelo no conjunto de testes usando várias métricas:<br>
AUC-ROC (Área sob a curva ROC)<br>
Acurácia: Mostra a porcentagem de predições corretas.<br>
Precisão (média macro)<br>
Revocação (Recall, média macro)<br>
F1 Score (média macro)<br>
Mean Squared Error (MSE)<br>


In [ ]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Set the parameters
params = {
  'objective': 'binary:logistic',
  'verbosity': 0,
  'booster': 'gbtree',
  'tree_method': 'gpu_hist',
  'learning_rate': 0.005171676107542148,
  'n_estimators': 593,
  'max_depth': 15,
  'subsample': 0.8068033937233334,
  'colsample_bytree': 0.9680836794118163,
}

model = xgb.XGBClassifier(**params)

# Fit the model to the training data
model.fit(X_train_pca, y_train)

# Predict the target values for the test data
y_proba = model.predict_proba(X_test_pca)[:, 1]
y_pred = model.predict(X_test_pca)

accuracy = accuracy_score(y_test, y_pred) * 100
precision = precision_score(y_test, y_pred, average='macro') * 100
recall = recall_score(y_test, y_pred, average='macro') * 100
f1 = f1_score(y_test, y_pred, average='macro') * 100
auc_roc = roc_auc_score(y_test, y_proba) * 100  # Notice we use y_proba here for AUC-ROC

# Print the scores
print(f"    Accuracy: {accuracy:.2f}%")
print(f"    Precision: {precision:.2f}%")
print(f"    Recall: {recall:.2f}%")
print(f"    F1-score: {f1:.2f}%")
print(f"    AUC-ROC: {auc_roc:.2f}%\n")


##### Importancia de cada Feacture

<b>Visualização da Importância dos Atributos com XGBoost</b>

Com o modelo treinado, uma característica importante do algoritmo XGBoost é a capacidade de visualizar a importância dos atributos. A importância de um atributo é calculada com base em quantas vezes o atributo foi usado em todas as árvores do modelo.

O método plot_importance do XGBoost nos permite visualizar essas importâncias em um formato gráfico

In [ ]:
plot_importance(model)
plt.show()